# Challenge Data Engineer

Este Notebook contiene las soluciones para resolver tres problemas usando la libreria de pyspark, consta de 4 secciones:
- Inicialización, en donde se importa las librerias, constantes y funciones que se usan en el resto del código
- Una sección por cada uno de los tres problemas del reto. Cada solución contiene la descripción, las soluciones con dos versiones, una para optimizar tiempo y otra para memoria y los resultados

## Inicialización

Ejecutar esta linea de codigo sin comentar (borrar #) solamente una vez al inicio de la sesión para instalar el package de emoji

In [2]:
#!pip install emoji

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, to_date, col, desc, row_number, udf, explode
from pyspark.sql.types import ArrayType, StringType
import pyspark.sql.functions as F

from pyspark.sql.window import Window

from typing import List, Tuple
import datetime
import emoji
import re

### Definición de constantes

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

Definir una función que genera la session de Spark u obtiene una versión ya existente

In [3]:
def get_spark_session() -> SparkSession:
    return (SparkSession.builder
            .appName("TwitterAnalysis")
            .getOrCreate())

## Problema 1
Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene
por cada uno de esos días. Debe incluir las siguientes funciones:
- def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
- def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:

### Solución q1_memory

In [4]:

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    spark = get_spark_session()

    date_format = "yyyy-MM-dd'T'HH:mm:ssXXX"
    
    tweetsDF = (spark.read.option("inferSchema","true")
                       .option("header","true")
                       .json(file_path)
                       .select("date", "user.username")
                       .withColumn("date", to_date(col("date"), date_format))
               )

    top_dates_df = (tweetsDF.groupBy("date")
                           .agg(count("*").alias("count"))
                           .orderBy(desc("count"))
                           .limit(10)
                   )


    top_dates_set = {row['date'] for row in top_dates_df.collect()}
    filtered_df = tweetsDF.filter(col("date").isin(top_dates_set))

    grouped_df = (filtered_df.groupBy("date", "username")
                             .agg(count("*").alias("count"))
                  )

    windowSpec = Window.partitionBy("date").orderBy(desc("count"))
    
    top_user_df = (grouped_df.withColumn("row_number", row_number().over(windowSpec))
                          .filter(col("row_number") == 1)
                          .drop("row_number")
                  )
    top_user_df = top_user_df.withColumnRenamed("count", "tweets_by_user")
    
    top_dates_with_users = (top_user_df.join(top_dates_df, "date")
                                   .orderBy(desc("count"), "date")
                           )

    return [(row['date'], row['username']) for row in top_dates_with_users.collect()]

Presentación de los resultados

In [5]:
print("----- memory -------")
q1_memory_result = q1_memory(file_path)
for e in q1_memory_result:
    print(e)

----- memory -------
(datetime.date(2021, 2, 12), 'RanbirS00614606')
(datetime.date(2021, 2, 13), 'MaanDee08215437')
(datetime.date(2021, 2, 17), 'RaaJVinderkaur')
(datetime.date(2021, 2, 16), 'jot__b')
(datetime.date(2021, 2, 14), 'rebelpacifist')
(datetime.date(2021, 2, 18), 'neetuanjle_nitu')
(datetime.date(2021, 2, 15), 'jot__b')
(datetime.date(2021, 2, 20), 'MangalJ23056160')
(datetime.date(2021, 2, 23), 'Surrypuria')
(datetime.date(2021, 2, 19), 'Preetm91')


### Solución q1_time

In [6]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    spark = get_spark_session()

    date_format = "yyyy-MM-dd'T'HH:mm:ssXXX"
    
    tweetsDF = (spark.read.option("inferSchema","true")
                       .option("header","true")
                       .json(file_path)
                       .select("date", "user.username")
                       .withColumn("date", to_date(col("date"), date_format))
               ).cache()  # Cache tweetsDF as it is used in multiple operations

    top_dates_df = (tweetsDF.groupBy("date")
                           .agg(count("*").alias("count"))
                           .orderBy(desc("count"))
                           .limit(10)
                   ).cache()  # Cache top_dates_df as it is used in multiple operations

    # Use broadcast join for smaller DataFrame to speed up join operation
    filtered_df = tweetsDF.join(F.broadcast(top_dates_df), "date")

    grouped_df = (filtered_df.groupBy("date", "username")
                             .agg(count("*").alias("count"))
                  ).cache()  # Cache grouped_df as it is used in multiple operations

    windowSpec = Window.partitionBy("date").orderBy(desc("count"))
    
    top_user_df = (grouped_df.withColumn("row_number", row_number().over(windowSpec))
                          .filter(col("row_number") == 1)
                          .drop("row_number")
                  ).withColumnRenamed("count", "tweets_by_user")
    
    top_dates_with_users = (top_user_df.join(top_dates_df, "date")
                                   .orderBy(desc("count"), "date")
                           )

    return [(row['date'], row['username']) for row in top_dates_with_users.collect()]

In [7]:
print("----- time -------")
q1_time_result = q1_time(file_path)
for e in q1_time_result:
    print(e)

----- time -------
(datetime.date(2021, 2, 12), 'RanbirS00614606')
(datetime.date(2021, 2, 13), 'MaanDee08215437')
(datetime.date(2021, 2, 17), 'RaaJVinderkaur')
(datetime.date(2021, 2, 16), 'jot__b')
(datetime.date(2021, 2, 14), 'rebelpacifist')
(datetime.date(2021, 2, 18), 'neetuanjle_nitu')
(datetime.date(2021, 2, 15), 'jot__b')
(datetime.date(2021, 2, 20), 'MangalJ23056160')
(datetime.date(2021, 2, 23), 'Surrypuria')
(datetime.date(2021, 2, 19), 'Preetm91')


## Problema 2
Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
- def q2_time(file_path: str) -> List[Tuple[str, int]]:
- def q2_memory(file_path: str) -> List[Tuple[str, int]]:

### Solución q2_time

Definir una función que extrae todos los emojis usando el paquete de emoji (instalado al incio del notebook) emoji.emoji_list

In [8]:
def extract_emojis(text):
    return [char['emoji'] for char in emoji.emoji_list(text)]

# Registrar la función como una UDF
extract_emojis_udf = udf(extract_emojis, ArrayType(StringType()))

In [9]:
def q2_general(file_path: str) -> List[Tuple[str, int]]:
    spark = get_spark_session()
    
    tweetsDF = (spark.read.option("inferSchema","true")
                       .option("header","true")
                       .json(file_path)
                       .select("content"))
   
    # Usar la UDF para extraer emojis y crear una nueva columna en el DataFrame
    tweets_with_emojis_df = tweetsDF.withColumn("emojis", extract_emojis_udf(tweetsDF["content"]))

    df_only_emojis = tweets_with_emojis_df.filter("size(emojis) > 0")

    # Mostrar resultados
    #tweets_with_emojis_df.select("emojis", "content").show(truncate=False)
    #df_only_emojis.select("emojis").show(truncate=False)

    top_emojis = (df_only_emojis.select(explode("emojis").alias("emoji"))
              .groupBy("emoji")
              .agg(count("*").alias("count"))
              .orderBy(desc("count"))
              .limit(10))
    #top_emojis.show(truncate=False)
    return [(row['emoji'], row['count']) for row in top_emojis.collect()]

Presentación de los resultados

In [ ]:
q2_general_result = q2_general(file_path)
for e in q2_general_result:
    print(e)

## Problema 3
El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@)
que registra cada uno de ellos. Debe incluir las siguientes funciones:
  - def q3_time(file_path: str) -> List[Tuple[str, int]]:
  - def q3_memory(file_path: str) -> List[Tuple[str, int]]:

###  Solución q3_time

Definir una función que extrae menciones usando expresiones regulares

In [ ]:
def extract_mentions(text):
    return re.findall(r"@\w+", text)

# Registrar la función como una UDF
extract_mentions_udf = udf(extract_mentions, ArrayType(StringType()))

In [ ]:
def q3_general(file_path: str) -> List[Tuple[str, int]]: 
    spark = get_spark_session()
    tweetsDF = (spark.read.option("inferSchema","true")
                       .option("header","true")
                       .json(file_path)
                       .select("content"))

    # Usar la UDF para extraer menciones y crear una nueva columna en el DataFrame
    df_with_mentions = tweetsDF.withColumn("mentions", extract_mentions_udf(tweetsDF["content"]))
    
    # Crear una nueva fila para cada mención usando explode y contar las ocurrencias de cada mención
    top_mentions = (df_with_mentions.select(explode("mentions").alias("mention"))
                    .groupBy("mention")
                    .agg(count("*").alias("count"))
                    .orderBy(desc("count"))
                    .limit(10))
    
    return [(row['mention'], row['count']) for row in top_mentions.collect()]

In [ ]:
q3_general_result = q3_general(file_path)
for e in q3_general_result:
    print(e)